In [1]:
!pip install contractions
# !pip install transformers
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import contractions
import string
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
# with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Duyen/data/ggvec.npy', 'wb') as f:
#     np.save(f,wv)

# Save the model to a file
wv.save('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Duyen/data/gg_wv')

In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Duyen/data/cleaned/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Duyen/data/cleaned/test.csv')
df_val = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Duyen/data/cleaned/val.csv')


In [6]:
df_train

,tweet_text,location,type,target,datetime,year,date,time
0,donbradshawntv marshallampsuk came assistance ...,Canada,wildfire,0,2016-05-06 16:54:00,2016,2016-05-06,16:54:00
1,red cross distributes fo mcmurray wildfire eva...,Canada,wildfire,1,2016-05-30 17:30:57,2016,2016-05-30,17:30:57
2,interesting insights shifting communications l...,Canada,wildfire,0,2016-04-30 23:18:34,2016,2016-04-30,23:18:34
3,globeandmail oil sands producers helping worke...,Canada,wildfire,0,2016-05-30 08:00:05,2016,2016-05-30,08:00:05
4,ottawa match red cross donations fo mcmurray w...,Canada,wildfire,0,2016-05-06 21:06:11,2016,2016-05-06,21:06:11
...,...,...,...,...,...,...,...,...
31181,https co mzh h uqh nepal cloud burst death tol...,Earth,hurricane,0,2017-08-14 03:32:29+00:00,2017,2017-08-14,03:32:29
31182,wear uniform days year rain shine flood draugh...,Hopes Peak,hurricane,0,2017-08-11 09:27:36+00:00,2017,2017-08-11,09:27:36
31183,flash flood town dad still wants water plants,"Boston, MA",flood,0,2017-08-02 20:44:29+00:00,2017,2017-08-02,20:44:29
31184,brahmaputra water level recedes assam flood to...,India,flood,0,2017-08-19 03:20:57+00:00,2017,2017-08-19,03:20:57


In [7]:
df_train['text_split'] = df_train['tweet_text'].str.split()
df_train = df_train.dropna()

df_test['text_split'] = df_test['tweet_text'].str.split()
df_test = df_test.dropna()

df_val['text_split'] = df_val['tweet_text'].str.split()
df_val = df_val.dropna()

In [8]:
df_train.isnull().sum()

tweet_text    0
location      0
type          0
target        0
datetime      0
year          0
date          0
time          0
text_split    0
dtype: int64

In [9]:
# Create an empty list to store the mean vectors
mean_vectors = []

# Iterate over each row of the DataFrame
for index, row in df_train.iterrows():
    # Convert each list of words into a numpy array of word embeddings
    word_embeddings = []
    for word in row['text_split']:
        try:
            word_embeddings.append(wv[word])
        except KeyError:
            pass
    if word_embeddings == []:
      mean_vector = np.array(wv['unk'])
    else:
      # Calculate the mean vector of the word embeddings along the first axis
      mean_vector = np.array(np.mean(np.array(word_embeddings), axis=0))
    # Append the mean vector to the list
    mean_vectors.append(mean_vector)

In [10]:
# Create an empty list to store the mean vectors test set
mean_vectors_test = []

# Iterate over each row of the DataFrame
for index, row in df_test.iterrows():
    # Convert each list of words into a numpy array of word embeddings
    word_embeddings_test = []
    for word in row['text_split']:
        try:
            word_embeddings_test.append(wv[word])
        except KeyError:
            pass
    if word_embeddings_test == []:
      mean_vector_test = np.array(wv['unk'])
    else:
      # Calculate the mean vector of the word embeddings along the first axis
      mean_vector_test = np.array(np.mean(np.array(word_embeddings_test), axis=0))
    # Append the mean vector to the list
    mean_vectors_test.append(mean_vector_test)

In [11]:
# Create an empty list to store the mean vectors test set
mean_vectors_val = []

# Iterate over each row of the DataFrame
for index, row in df_val.iterrows():
    # Convert each list of words into a numpy array of word embeddings
    word_embeddings_val = []
    for word in row['text_split']:
        try:
            word_embeddings_val.append(wv[word])
        except KeyError:
            pass
    if word_embeddings_val == []:
      word_embeddings_val = np.array(wv['unk'])
    else:
      # Calculate the mean vector of the word embeddings along the first axis
      mean_vector_val = np.array(np.mean(np.array(word_embeddings_val), axis=0))
    # Append the mean vector to the list
    mean_vectors_val.append(mean_vectors_val)

In [12]:
X_train = mean_vectors
y_train = df_train['target']
X_test = mean_vectors_test
y_test = df_test['target']

# Use SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [13]:
#Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Make predictions on validation set
y_pred_rf = clf.predict(X_test)

precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)

In [14]:
# SVM
clf_svm = make_pipeline(LinearSVC(random_state=0, tol=1e-5))
clf_svm.fit(X_train_resampled, y_train_resampled)

y_pred_svm = clf_svm.predict(X_test)

precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

In [15]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense, Activation

# convert input data from list to numpy array
X_train_resampled = np.array(X_train_resampled)
y_train_resampled = np.array(y_train_resampled)
X_test = np.array(X_test)

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=X_train_resampled.shape[1]))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred_fnn = model.predict(X_test)
# y_pred_fnn = np.argmax(y_pred_fnn, axis=1)

Epoch 1/20
1274/1274 [==============================] - 5s 3ms/step - loss: 0.3722 - accuracy: 0.8375 - val_loss: 0.3647 - val_accuracy: 0.8446
Epoch 2/20
1274/1274 [==============================] - 3s 3ms/step - loss: 0.3075 - accuracy: 0.8691 - val_loss: 0.2667 - val_accuracy: 0.9035
Epoch 3/20
1274/1274 [==============================] - 3s 3ms/step - loss: 0.2795 - accuracy: 0.8840 - val_loss: 0.1993 - val_accuracy: 0.9362
Epoch 4/20
1274/1274 [==============================] - 11s 9ms/step - loss: 0.2564 - accuracy: 0.8944 - val_loss: 0.2037 - val_accuracy: 0.9298
Epoch 5/20
1274/1274 [==============================] - 3s 3ms/step - loss: 0.2374 - accuracy: 0.9049 - val_loss: 0.1428 - val_accuracy: 0.9576
Epoch 6/20
1274/1274 [==============================] - 3s 3ms/step - loss: 0.2192 - accuracy: 0.9131 - val_loss: 0.1925 - val_accuracy: 0.9316
Epoch 7/20
1274/1274 [==============================] - 3s 3ms/step - loss: 0.2038 - accuracy: 0.9199 - val_loss: 0.2298 - val_accuracy

In [16]:
y_pred_binary = (y_pred_fnn > 0.5).astype(int)
# y_pred_binary

In [17]:
precision_fnn = precision_score(y_test, y_pred_binary)
recall_fnn = recall_score(y_test, y_pred_binary)
f1_fnn = f1_score(y_test, y_pred_binary)
roc_auc_fnn = roc_auc_score(y_test, y_pred_binary)

In [18]:
data = {'Model': ['Random Forest', 'Support Vector Machine', 'Feedforward Neural Network'],
        'Precision': [precision_rf, precision_svm, precision_fnn],
        'Recall': [recall_rf, recall_svm, recall_fnn],
        'F1': [f1_rf, f1_svm, f1_fnn],
        'ROC-AUC': [roc_auc_rf, roc_auc_svm, roc_auc_fnn]}

df = pd.DataFrame(data)
df = df.set_index('Model')

print(df)


                            Precision    Recall        F1   ROC-AUC
Model                                                              
Random Forest                0.730233  0.647156  0.686189  0.778145
Support Vector Machine       0.652018  0.825639  0.728629  0.829081
Feedforward Neural Network   0.713690  0.775763  0.743433  0.828740


In [26]:
import pickle

# Assuming that you have a trained model called "model"
# Save the model to a file called "model.pkl"
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/svm_classification.pkl', 'wb') as f:
    pickle.dump(clf_svm, f)

# Load the saved model from the file
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/svm_classification.pkl', 'rb') as f:
    model_svm = pickle.load(f)

# Assuming that you have a trained model called "model"
# Save the model to a file called "model.pkl"
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/rf_classification.pkl', 'wb') as f:
    pickle.dump(clf, f)

# Load the saved model from the file
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/rf_classification.pkl', 'rb') as f:
    model_rf = pickle.load(f)

# Assuming that you have a trained model called "model"
# Save the model to a file called "model.pkl"
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/fnn_classification.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the saved model from the file
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/fnn_classification.pkl', 'rb') as f:
    model_fnn = pickle.load(f)    

In [60]:
new_df1 = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Apify Uploads/dataset.csv')
new_df1 = new_df1[['created_at','full_text']]
new_df2 = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Apify Uploads/dataset (1).csv')
new_df2 = new_df2[['created_at','full_text']]
new_df3 = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Apify Uploads/dataset_twitter-latest-scraper_2023-04-20_17-11-15-498.csv')
new_df3 = new_df3[['created_at','full_text']]
new_df = pd.concat([new_df1, new_df2, new_df3], axis=0)

In [21]:
# new_df.to_csv('/content/drive/MyDrive/DSCI560/concatenated_df.csv')

In [61]:
stopwords_english = stopwords.words('english')
# convert to lower case
new_df['full_text'] = new_df['full_text'].str.lower()

# remove the HTML and URLs from the reviews (ref: https://stackoverflow.com/questions/45999415/removing-html-tags-in-pandas)
new_df['full_text'] = new_df['full_text'].str.replace(r'<[^<>]*>', '', regex=True)

# remove non-alphabetical characters and punctuations from all tweets.
# print(string.punctuation)
new_df['full_text'] = new_df['full_text'].str.replace('[^a-zA-Z]', ' ').replace('[{}]'.format(string.punctuation), '')

# remove extra spaces from all reviews.
new_df['full_text'] = new_df['full_text'].str.strip()

# perform contractions on the reviews by using the package contractions (ref: https://www.dataquest.io/blog/how-to-clean-and-prepare-your-data-for-analysis/).
# new_df['full_text'] = new_df['full_text'].astype(str).apply(lambda x: contractions.fix(x))

#remove the stop words (ref: https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe)
new_df['full_text'] = new_df['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_english)]))

new_df['text_split'] = new_df['full_text'].str.split()
new_df = new_df.dropna()

In [62]:
new_df.shape

(250, 3)

In [63]:
# Create an empty list to store the mean vectors
mean_vectors_infer = []

# Iterate over each row of the DataFrame
for index, row in new_df.iterrows():
    # Convert each list of words into a numpy array of word embeddings
    word_embeddings_infer = []
    for word in row['text_split']:
        try:
            word_embeddings_infer.append(wv[word])
        except KeyError:
            pass
    if word_embeddings_infer == []:
      mean_vector_infer = np.array(wv['unk'])
    else:
      # Calculate the mean vector of the word embeddings along the first axis
      mean_vector_infer = np.array(np.mean(np.array(word_embeddings_infer), axis=0))
    # Append the mean vector to the list
    mean_vectors_infer.append(mean_vector_infer)

In [64]:
X_test_infer = mean_vectors_infer

# Use the loaded model to make predictions on the new dataset
predictions_svm = model_svm.predict(X_test_infer)
predictions_rf = model_rf.predict(X_test_infer)
# predictions_fnn = model_fnn.predict(X_test)

In [65]:
X_test_infer_ = np.array(X_test_infer)

In [66]:
predictions_fnn = model_fnn.predict(X_test_infer_)
predictions_fnn = (predictions_fnn > 0.5).astype(int)


8/8 [==============================] - 0s 3ms/step


In [76]:
# predictions_fnn

In [73]:
flattened_arr = predictions_fnn.ravel()


tweet_dict1 = {'tweet_text': new_df['full_text'], 'label': predictions_svm}
tweet_dict2 = {'tweet_text': new_df['full_text'], 'label': predictions_rf}
tweet_dict3 = {'tweet_text': new_df['full_text'], 'label': flattened_arr}

# Convert the dictionary to a pandas dataframe
tweet_df1 = pd.DataFrame(tweet_dict1)
tweet_df2 = pd.DataFrame(tweet_dict2)
tweet_df3 = pd.DataFrame(tweet_dict3)


tweet_df = pd.concat([tweet_df1, tweet_df2, tweet_df3], axis=0)

# Filter the dataframe to only include tweets with a label of 1
disaster_df = tweet_df[tweet_df['label'] == 1].reset_index(drop=True)

In [75]:
disaster_df.drop_duplicates(subset=['tweet_text', 'label']).reset_index(drop=True)

,tweet_text,label
0,earthquake magnitudo samoa islands region minu...,1
1,,1
2,marcorubio cbsmiami notice floods useless gove...,1
3,sti spreading like wildfire flood victims evac...,1
4,mikehudema stop building flood zones,1
5,mejenwalton dutchbros nicely done,1
6,salvalucania icardiyolog ilkertiano ultrasturk...,1
7,marquette county declares local state emergenc...,1
8,stillgray men dressing painting like women con...,1
9,sorunun sorunu flood,1
